In [1]:
import os
import pandas as pd
from collections import defaultdict
from functools import partial
from multiprocessing import cpu_count
from pathlib import Path
from textwrap import dedent
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import Dataset
import torch as tf
from collections import Counter
from datetime import datetime
import math
from sklearn.metrics import confusion_matrix
from tools import load_sensor_data_without_h, sample_sensor_data, manual_lable_array_list, most_frequent, extract_sensor_data, get_sensor_data, sample_data, sample_label, combine_to_one
from model import Classifier_dh

In [ ]:
txt_to_label = {'talk':0, 'eat':1, 'read':2, 'drink':3, 'computer':4, 'write':5, 'other': 6}
actor_1 = [1,2,3,4,5,6,7,8,9,10]
actor_2 = [11,12,13,14,31,32,33,34,35,36]
actor_3 = [15,16,17,18,37,38,39,40,41,42]
actor_4 = [43,44,45,46,47,48,49,50,51,52]
actor_5 = [53,54,55,56,57,58,59,60,61,62]
actor_6 = [63,64,65,66,67,68,69,70,71,72]
actor_7 = [76,77,78,79,80,81,82,83,84,85]
actor_8 = [86,87,88,89,90,91,92,93,94,95]
actor_9 = [97,98,99,100,101,102,103,104,105,106]
actor_10 = [107,108,109,110,111,112,113,114,115,116]
actor_11 = [117,118,119,120,121,122,123,124,125,126]
actor_12 = [128,129,130,131,132,133,134,135,136,137]
actor_13 = [138,139,140,141,142,143,144,145,146,147]
actor_14 = [148,149,150,151,152,153,154,155,156,157]
actor_15 = [161,162,163,164,165,166,167,168,169,170]
actor_16 = [171,172,173,174,175,176,177,178,179,180]
actor_17 = [181,182,183,184,185,186,187,188,189,190]
actor_18 = [191,192,193,194,195,196,197,198,199,200]
actor_19 = [201,202,203,204,205,206,207,208,209,210]
actor_20 = [212,213,214,215,216,217,218,219,220,221]
actor_21 = [223,224,225,226,227,228,229,230,231,232]
actor_22 = [233,234,235,236,237,238,239,240,241,242]
actor_23 = [243,244,245,246,247,248,249,250,251,252]
actor_24 = [254,255,256,257,258,259,260,261,262,263]
actor_25 = [264,265,266,267,268,269,270,271,272,273]
combine_list = actor_1 + actor_2 + actor_3 + actor_4 + actor_5 + actor_6 + actor_7 + actor_8 + actor_9 + actor_10+ actor_11 + actor_12 + actor_13 + actor_14 + actor_15+ actor_16 + actor_17 + actor_18 + actor_19 + actor_20+ actor_21 + actor_22 + actor_23 + actor_24 + actor_25
window_sz = 128
sample_sz = 128
lr = 0.001
n_epochs = 500
num_classes = 6
patience, trials = 100, 0
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss(reduction='sum')
bs = 256

In [3]:
#manual_lable_array_list('../manual_lable.csv', combine_list)
#extract_sensor_data(combine_list) 

#label_list = np.load('../save_data/label_data/label_list.npy', allow_pickle=True).item()
#sensor_data = get_sensor_data('../save_data/original_data/')

#sample_data(sensor_data, combine_list, window_sz = window_sz, sample_sz = sample_sz)
#sample_label(label_list, combine_list, window_sz = window_sz, sample_sz = sample_sz)
   
sample_sensor_data = get_sensor_data('../save_data/sample_data/')
sample_sensor_label = get_sensor_data('../save_data/sample_label/')

In [4]:
total_num_list = total_num_list = actor_1 + actor_2 + actor_3 + actor_4+actor_5+actor_6+actor_7+actor_8+actor_9+actor_10+actor_11+actor_12+actor_13+actor_14+actor_15+actor_16+actor_17+actor_18+actor_19+actor_20+actor_21+actor_22+actor_23+actor_24+actor_25
val_num_list = total_num_list[1*10 : 1*10+10]
del total_num_list[1*10 : 1*10+10]
train_num_list = total_num_list
train_sensor_data = combine_to_one(sample_sensor_data,train_num_list)
val_sensor_data = combine_to_one(sample_sensor_data,val_num_list)
train_sensor_label = combine_to_one(sample_sensor_label,train_num_list)
val_sensor_label = combine_to_one(sample_sensor_label,val_num_list)
train_ds = TensorDataset(torch.tensor(train_sensor_data).float(), torch.tensor(train_sensor_label).long())
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=0)
val_ds = TensorDataset(torch.tensor(val_sensor_data).float(), torch.tensor(val_sensor_label).long())
val_dl = DataLoader(val_ds, batch_size=bs, shuffle=True, num_workers=0)

In [16]:
train_ds = TensorDataset(torch.tensor(train_sensor_data).float(), torch.tensor(train_sensor_label).long())
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=0)
val_ds = TensorDataset(torch.tensor(val_sensor_data).float(), torch.tensor(val_sensor_label).long())
val_dl = DataLoader(val_ds, batch_size=bs, shuffle=True, num_workers=0)

In [17]:
class LSTMClassifier(nn.Module):
    """Very simple implementation of LSTM-based time-series classifier."""
    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.batch_size = None
        self.hidden = None
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.cuda() for t in (h0, c0)]

In [18]:
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [19]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

In [20]:
from torch.optim.lr_scheduler import _LRScheduler

In [21]:
input_dim = 39   
hidden_dim = 256
layer_dim = 3
output_dim = 6
seq_dim = 128

lr = 0.001
n_epochs = 1000
iterations_per_epoch = len(train_dl)
best_acc = 0
patience, trials = 100, 0

model = LSTMClassifier(input_dim, hidden_dim, layer_dim, output_dim)
model = model.cuda()
criterion = nn.CrossEntropyLoss(reduction='sum')
opt = torch.optim.RMSprop(model.parameters(), lr=lr)
sched = CyclicLR(opt, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    for i, (x_batch, y_batch) in enumerate(train_dl):
        model.train()
        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()
        y_batch = tf.squeeze(y_batch)
        #sched.step()
        opt.zero_grad()
        out = model(x_batch)
        #print(out.shape)
        #print(y_batch.shape)
        loss = criterion(out, y_batch)
        loss.backward()
        opt.step()
    
    model.eval()
    correct, total = 0, 0
    for x_val, y_val in val_dl:
        x_val, y_val = [t.cuda() for t in (x_val, y_val)]
        y_val = tf.squeeze(y_val)
        out = model(x_val)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_val.size(0)
        correct += (preds == y_val).sum().item()
    
    acc = correct / total

    if epoch % 5 == 0:
        print(f'Epoch: {epoch:3d}. Loss: {loss.item():.4f}. Acc.: {acc:2.2%}')

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break

Start model training
Epoch 1 best model saved with accuracy: 34.52%
Epoch 3 best model saved with accuracy: 39.43%
Epoch:   5. Loss: 47.4997. Acc.: 24.91%
Epoch:  10. Loss: 56.8568. Acc.: 33.27%
Epoch:  15. Loss: 34.9258. Acc.: 28.43%
Epoch:  20. Loss: 18.9336. Acc.: 36.58%
Epoch:  25. Loss: 4.6144. Acc.: 31.92%
Epoch:  30. Loss: 9.2812. Acc.: 34.34%
Epoch:  35. Loss: 11.4466. Acc.: 32.35%
Epoch:  40. Loss: 4.0585. Acc.: 41.17%
Epoch 40 best model saved with accuracy: 41.17%
Epoch:  45. Loss: 8.7170. Acc.: 33.49%
Epoch:  50. Loss: 5.7565. Acc.: 33.81%
Epoch:  55. Loss: 3.9265. Acc.: 32.92%
Epoch:  60. Loss: 5.8446. Acc.: 29.68%


KeyboardInterrupt: 

In [4]:
bestacc = []
for actor in range(24,25):
    total_num_list = total_num_list = actor_1 + actor_2 + actor_3 + actor_4+actor_5+actor_6+actor_7+actor_8+actor_9+actor_10+actor_11+actor_12+actor_13+actor_14+actor_15+actor_16+actor_17+actor_18+actor_19+actor_20+actor_21+actor_22+actor_23+actor_24+actor_25
    val_num_list = total_num_list[actor*10 : actor*10+10]
    del total_num_list[actor*10 : actor*10+10]
    train_num_list = total_num_list
    train_sensor_data = combine_to_one(sample_sensor_data,train_num_list)
    val_sensor_data = combine_to_one(sample_sensor_data,val_num_list)
    train_sensor_label = combine_to_one(sample_sensor_label,train_num_list)
    val_sensor_label = combine_to_one(sample_sensor_label,val_num_list)
    model = Classifier_dh(train_sensor_data.shape[1], train_sensor_data.shape[2], num_classes).to(device)
    opt = optim.Adam(model.parameters(), lr=lr)
    train_ds = TensorDataset(torch.tensor(train_sensor_data).float(), torch.tensor(train_sensor_label).long())
    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=0)
    val_ds = TensorDataset(torch.tensor(val_sensor_data).float(), torch.tensor(val_sensor_label).long())
    val_dl = DataLoader(val_ds, batch_size=bs, shuffle=True, num_workers=0)
    best_acc = 0
    loss_history = []
    acc_history = []
    print('Start model training')
    for epoch in range(1, n_epochs + 1):
        model.train()
        epoch_loss = 0
        for i, batch in enumerate(train_dl):
            x_raw, y_batch = [t.to(device) for t in batch]
            y_batch = tf.squeeze(y_batch)
            opt.zero_grad()
            out = model(x_raw)
            loss = criterion(out, y_batch)
            epoch_loss += loss.item()
            loss.backward()
            opt.step()
        loss_history.append(epoch_loss)
        model.eval()
        correct, total = 0, 0
        for batch in val_dl:
            x_raw, y_batch = [t.to(device) for t in batch]
            y_batch = tf.squeeze(y_batch)
            out = model(x_raw)
            preds = F.log_softmax(out, dim=1).argmax(dim=1)
            if preds.size()[0] > 1:
                total += y_batch.size(0)
                correct += (preds == y_batch).sum().item()
        acc = correct / total
        acc_history.append(acc)
        if epoch % 10 == 0:
            print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
        if acc > best_acc:
            trials = 0
            best_acc = acc
            torch.save(model.state_dict(), 'best.pth')
            print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
        else:
            trials += 1
            if trials >= patience:
                print(f'Early stopping on epoch {epoch}')
                break
    bestacc.append(best_acc)

Start model training
Epoch 1 best model saved with accuracy: 26.33%
Epoch 2 best model saved with accuracy: 33.91%
Epoch 7 best model saved with accuracy: 34.06%
Epoch 8 best model saved with accuracy: 34.45%
Epoch:  10. Loss: 53380.4737. Acc.: 34.70%
Epoch 10 best model saved with accuracy: 34.70%
Epoch 11 best model saved with accuracy: 37.47%
Epoch:  20. Loss: 43439.9625. Acc.: 36.41%
Epoch:  30. Loss: 39022.7170. Acc.: 34.38%
Epoch:  40. Loss: 36165.1663. Acc.: 33.13%
Epoch:  50. Loss: 34818.9941. Acc.: 34.20%
Epoch:  60. Loss: 33508.0131. Acc.: 37.05%
Epoch:  70. Loss: 32642.9418. Acc.: 34.52%
Epoch:  80. Loss: 31990.1024. Acc.: 35.12%
Epoch:  90. Loss: 31362.4975. Acc.: 33.49%
Epoch: 100. Loss: 30761.7552. Acc.: 34.38%
Epoch: 110. Loss: 30452.2671. Acc.: 34.20%
Early stopping on epoch 111


In [5]:
bestacc

[0.3747330960854093]

In [8]:
A=[39.86, 54.80, 67.65, 44.23, 69.04, 65.87, 70.82, 53.27, 51.71, 69.40, 68.90, 59.82, 56.69, 69.07, 62.38, 60.32, 56.98, 73.74, 51.07, 64.84, 58.97, 51.25, 48.26, 52.63, 37.47]

In [9]:
sum(A)/25

58.361599999999996

In [1]:
A=[54.80, 67.65, 44.23, 69.04, 65.87, 70.82, 53.27, 51.71, 69.40, 68.90, 59.82, 56.69, 69.07, 62.38, 60.32, 56.98, 73.74, 51.07, 64.84, 58.97, 51.25, 48.26, 52.63]

In [2]:
sum(A)/23

60.07434782608696